In [1]:
from config import config as cfg
from preprocess import *
import numpy as np
import wandb
import torch
import pandas as pd
import os

In [2]:
if torch.cuda.is_available():
    DEVICE = torch.device('cuda')
else:
    DEVICE = torch.device('cpu')
print(DEVICE)

cuda


In [3]:
DEV_TRAIN_LEN = cfg['parameters']['dev_train_len']['value']
DEV_VALIDATION_LEN = cfg['parameters']['dev_validation_len']['value']

DIR = '/scratch/shu7bh/RES/PRE'

In [4]:
import os
if not os.path.exists(DIR):
    os.makedirs(DIR)

In [5]:
print(DEV_TRAIN_LEN)
print(DEV_VALIDATION_LEN)

5000
1000


In [6]:
df = pd.read_csv('data/train.csv')
df = df.sample(frac=1, random_state=0).reset_index(drop=True)
df['Description'] = df['Description'].apply(tokenize_corpus)
df['Description'] = df['Description'].apply(get_word_tokenized_corpus)

In [7]:
dev_train = df[:DEV_TRAIN_LEN]['Description']
dev_validation = df[DEV_TRAIN_LEN:DEV_TRAIN_LEN + DEV_VALIDATION_LEN]['Description']

In [8]:
dev_validation

5000    [video, game, publisher, electronic, arts, on,...
5001    [by, amanda, gardner, ,, healthday, reporter, ...
5002    [percival, became, the, tigers, 39, ;, new, cl...
5003    [it, 's, getting, harder, to, shrink, chips, ,...
5004    [canadian, press, halifax, cp, they, have, bec...
                              ...                        
5995    [massive, database, holds, info, on, millions,...
5996    [supreme, court, justices, on, tuesday, uncork...
5997    [the, yankees, should, soon, clinch, their, se...
5998    [the, un, nuclear, agency, agreed, yesterday, ...
5999    [ap, how, do, you, explain, a, quarterback, sn...
Name: Description, Length: 1000, dtype: object

In [9]:
from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader
from dataset import SentencesDataset
from elmo import ELMO
from torch import nn

In [10]:
class Collator:
    def __init__(self, Emb):
        self.pad_index = Emb.key_to_index['<pad>']

    def __call__(self, batch):
        X, X_lengths = zip(*batch)
        X = pad_sequence(X, batch_first=True, padding_value=self.pad_index)
        return X[:, :-1], X[:, 1:], torch.stack(X_lengths) - 1

In [11]:
import tqdm

def fit(model, dataloader, train, es, loss_fn, optimizer):
    model.train() if train else model.eval()
    epoch_loss = []

    pbar = tqdm.tqdm(dataloader)

    for X, Y, X_lengths in pbar:
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)

        Y_pred = model(X, X_lengths)
        Y_pred = Y_pred.reshape(-1, Y_pred.shape[2])

        Y = Y.reshape(-1)

        loss = loss_fn(Y_pred, Y)
        epoch_loss.append(loss.item())

        X.detach()
        Y_pred.detach()
        Y.detach()

        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        pbar.set_description(f'{"T" if train else "V"} Loss: {loss.item():7.4f}, Avg Loss: {np.mean(epoch_loss):7.4f}, Best Loss: {es.best_loss:7.4f}, Counter: {es.counter}')

    return np.mean(epoch_loss)

In [12]:
import EarlyStopping as ES

def train(EPOCHS, elmo, training_dataloader, validation_dataloader, loss_fn, optimizer):
    es = ES.EarlyStopping(patience=2, delta=0.01)

    for epoch in range(EPOCHS):
        print(f'\nEpoch {epoch+1}')

        epoch_loss = fit(elmo, training_dataloader, True, es, loss_fn, optimizer)
        wandb.log({'train_loss': epoch_loss})

        with torch.no_grad():
            epoch_loss = fit(elmo, validation_dataloader, False, es, loss_fn, optimizer)
            wandb.log({'validation_loss': epoch_loss})
            if es(epoch_loss, epoch):
                break
            if es.counter == 0:
                torch.save(elmo.state_dict(), os.path.join(DIR, f'best_model.pth'))
    
        # torch.save(elmo.state_dict(), os.path.join(DIR, f'elmo_{epoch + 1}.pth'))

    wandb.log({'loss': es.best_loss})
    # os.rename(os.path.join(DIR, f'elmo_{es.best_model_pth + 1}.pth'), os.path.join(DIR, 'best_model.pth'))

    best_model = wandb.Artifact(f'best_model_{wandb.run.id}', type='model')
    best_model.add_file(os.path.join(DIR, 'best_model.pth'))
    wandb.run.log_artifact(best_model)
    # wandb.save(os.path.join(DIR, 'best_model.pth'))

In [13]:
def run(cfg=None):
    with wandb.init(config=cfg):
        config = wandb.config
        BATCH_SIZE = 16
        if config.hidden_dim in [300, 500]:
            BATCH_SIZE = 32

        wandb.log({'batch_size': BATCH_SIZE})
        HIDDEN_DIM = config['hidden_dim']
        DROP_OUT = config['dropout']
        OPTIMIZER = config['optimizer']
        LEARNING_RATE = config['learning_rate']
        EPOCHS = config['epochs']
        EMBEDDITNG_DIM = config['embedding_dim']
        NUM_LAYERS = config['num_layers']

        Emb = create_vocab(df['Description'], EMBEDDITNG_DIM)

        dev_train_dataset = SentencesDataset(dev_train, Emb)
        dev_validation_dataset = SentencesDataset(dev_validation, Emb)

        collate_fn = Collator(Emb)

        training_dataloader = DataLoader(dev_train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=4)
        validation_dataloader = DataLoader(dev_validation_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=4)

        torch.cuda.empty_cache()

        elmo = ELMO(Emb, HIDDEN_DIM, DROP_OUT, NUM_LAYERS).to(DEVICE)

        optimizer = getattr(torch.optim, OPTIMIZER)(elmo.parameters(), lr=LEARNING_RATE)
        loss_fn = nn.CrossEntropyLoss(ignore_index=Emb.key_to_index['<pad>'])

        train(EPOCHS, elmo, training_dataloader, validation_dataloader, loss_fn, optimizer)

sweep_id = wandb.sweep(cfg, project='ELMO')
wandb.agent(sweep_id, run, count=1)
# wandb.agent(sweep_id, run_sweep, count=20)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: vtpegz9o
Sweep URL: https://wandb.ai/shu7bh/ELMO/sweeps/vtpegz9o


wandb: Agent Starting Run: q9tp7d56 with config:
wandb: 	dev_train_len: 5000
wandb: 	dev_validation_len: 1000
wandb: 	dropout: 0
wandb: 	embedding_dim: 50
wandb: 	epochs: 100
wandb: 	hidden_dim: 100
wandb: 	learning_rate: 0.001
wandb: 	num_layers: 2
wandb: 	optimizer: Adam
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shu7bh. Use `wandb login --relogin` to force relogin



Epoch 1


T Loss:  6.7581, Avg Loss:  7.4066, Best Loss:     inf, Counter: 0: 100%|██████████| 313/313 [00:07<00:00, 39.36it/s]
V Loss:  6.9564, Avg Loss:  7.0115, Best Loss:     inf, Counter: 0: 100%|██████████| 63/63 [00:00<00:00, 68.21it/s]


...


Epoch 43


T Loss:  0.0068, Avg Loss:  0.0053, Best Loss:  1.4452, Counter: 2: 100%|██████████| 313/313 [00:07<00:00, 41.19it/s]
V Loss:  1.5720, Avg Loss:  1.4635, Best Loss:  1.4452, Counter: 2: 100%|██████████| 63/63 [00:00<00:00, 67.17it/s]



Epoch 44


T Loss:  0.0065, Avg Loss:  0.0101, Best Loss:  1.4452, Counter: 3: 100%|██████████| 313/313 [00:07<00:00, 41.37it/s]
V Loss:  1.1863, Avg Loss:  1.4571, Best Loss:  1.4452, Counter: 3: 100%|██████████| 63/63 [00:00<00:00, 68.57it/s]


batch_size,▁
loss,▁
train_loss,█▇▆▅▄▄▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
validation_loss,█▇▆▅▄▃▃▃▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
batch_size,16
loss,1.44521
train_loss,0.01009
validation_loss,1.45711


In [22]:
# import wandb
# wrun = wandb.init()
# artifact = wrun.use_artifact('shu7bh/ELMO/best_model_3y4z0vh5:v0', type='model')
# cfg = wrun.config
# artifact_dir = artifact.download()

wrun = wandb.Api().artifact('shu7bh/ELMO/best_model_q9tp7d56:v0', type='model')
artifact_dir = wrun.download()
# shu7bh/ELMO/runs/q9tp7d56
cfg = wandb.Api().run('shu7bh/ELMO/q9tp7d56').config

wandb: Downloading large artifact best_model_q9tp7d56:v0, 54.45MB. 1 files... 
wandb:   1 of 1 files downloaded.  
Done. 0:0:1.2


In [23]:
import tqdm

def run_epoch(model, dataloader, loss_fn):
    epoch_loss = []

    pbar = tqdm.tqdm(dataloader)

    for X, Y, X_lengths in pbar:
        X = X.to(DEVICE)
        Y = Y.to(DEVICE)

        Y_pred = model(X, X_lengths)
        Y_pred = Y_pred.reshape(-1, Y_pred.shape[2])

        Y = Y.reshape(-1)

        loss = loss_fn(Y_pred, Y)
        epoch_loss.append(loss.item())

        pbar.set_description(f'Loss: {loss.item():7.4f}, Avg Loss: {np.mean(epoch_loss):7.4f}')

    return np.mean(epoch_loss)

In [24]:
def validate(elmo, validation_dataloader, loss_fn):
    with torch.no_grad():
        elmo.eval()
        epoch_loss = run_epoch(elmo, validation_dataloader, loss_fn)
        print(f'Validation Loss: {epoch_loss:7.4f}')

In [25]:
Emb = create_vocab(df['Description'], cfg['embedding_dim'])

# dev_train_dataset = SentencesDataset(dev_train, Emb)
dev_validation_dataset = SentencesDataset(dev_validation, Emb)

collate_fn = Collator(Emb)

# training_dataloader = DataLoader(dev_train_dataset, batch_size=BATCH_SIZE, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=4)
validation_dataloader = DataLoader(dev_validation_dataset, batch_size=16, shuffle=True, collate_fn=collate_fn, pin_memory=True, num_workers=4)
elmo = ELMO(Emb, cfg['hidden_dim'], cfg['dropout'], cfg['num_layers']).to(DEVICE)

elmo.load_state_dict(torch.load(os.path.join(artifact_dir, 'best_model.pth')))

validate(elmo, validation_dataloader, nn.CrossEntropyLoss(ignore_index=Emb.key_to_index['<pad>']))

Loss:  1.8123, Avg Loss:  1.4573: 100%|██████████| 63/63 [00:00<00:00, 70.66it/s]

Validation Loss:  1.4573
